In [56]:
!gdown 19xU3kUyivRWGQzYXnpwlIjQqdfkD56v4

Downloading...
From: https://drive.google.com/uc?id=19xU3kUyivRWGQzYXnpwlIjQqdfkD56v4
To: /content/twitter_data.csv
100% 10.3M/10.3M [00:00<00:00, 133MB/s]


In [57]:
!pip show tensorflow

Name: tensorflow
Version: 2.19.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tensorflow_decision_forests, tf_keras


In [58]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
import nltk
from nltk.stem import WordNetLemmatizer

In [59]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [60]:
data = pd.read_csv('/content/twitter_data.csv')
data

,id,user,sentiment,tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
...,...,...,...,...
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...


In [61]:
data.isnull().sum()

,0
id,0
user,0
sentiment,0
tweet,686


In [62]:
data = data.dropna(axis=0)

In [63]:
num_classes = len(Y.columns)
num_classes

4

In [64]:
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = ' '.join([lemmatizer.lemmatize(w) for w in text.split()])
    return text

In [65]:
data['tweet'] = data['tweet'].apply(clean_text)

/tmp/ipython-input-2420922662.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet'] = data['tweet'].apply(clean_text)


In [66]:
X = data['tweet']
Y = pd.get_dummies(data['sentiment'])

In [67]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y,
    test_size=0.2,
    random_state=42
)

In [68]:
max_tokens = 10000
maxlen = 64

tokenizer = Tokenizer(num_words=max_tokens)
tokenizer.fit_on_texts(X_train.values)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (59196, 64)
Test shape: (14800, 64)


In [69]:
from keras.models import Sequential
import keras
from keras.layers import (
    Dense,
    Embedding,
    Conv1D,
    MaxPooling1D,
    LSTM,
    Flatten,
    Bidirectional,
    GlobalMaxPooling1D,
    Dropout,
    BatchNormalization
)
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [70]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2026-02-22 19:55:20--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2026-02-22 19:55:20--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2026-02-22 19:55:21--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

In [79]:
embeddings_index = {}

with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.array(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print(f"Total words in GloVe: {len(embeddings_index)}")

Total words in GloVe: 400000


In [80]:
embedding_dim = 100

embedding_matrix = np.zeros((max_tokens, embedding_dim))

for word, i in tokenizer.word_index.items():
    if i < max_tokens:
        vec = embeddings_index.get(word)
        if vec is not None:
            embedding_matrix[i] = vec

print("Embedding matrix shape:", embedding_matrix.shape)

Embedding matrix shape: (10000, 100)


In [81]:
model = Sequential()

model.add(Embedding(
    input_dim=max_tokens,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    trainable=True
))

model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=0.4, recurrent_dropout=0.3)))

model.add(GlobalMaxPooling1D())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


In [82]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

history = model.fit(
    X_train, Y_train,
    validation_split=0.2,
    batch_size=64,
    epochs=50,
    callbacks=[early_stopping, lr_scheduler]
)

Epoch 1/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 125s 154ms/step - accuracy: 0.3518 - loss: 1.3441 - precision_5: 0.4564 - recall_5: 0.0374 - val_accuracy: 0.5474 - val_loss: 1.0917 - val_precision_5: 0.6654 - val_recall_5: 0.3894 - learning_rate: 0.0010
Epoch 2/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 141s 153ms/step - accuracy: 0.5522 - loss: 1.1072 - precision_5: 0.6833 - recall_5: 0.3401 - val_accuracy: 0.6085 - val_loss: 0.9743 - val_precision_5: 0.7014 - val_recall_5: 0.5163 - learning_rate: 0.0010
Epoch 3/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 113s 153ms/step - accuracy: 0.6222 - loss: 0.9555 - precision_5: 0.7513 - recall_5: 0.4658 - val_accuracy: 0.6500 - val_loss: 0.8742 - val_precision_5: 0.7711 - val_recall_5: 0.5247 - learning_rate: 0.0010
Epoch 4/50
740/740 ━━━━━━━━━━━━━━━━━━━━ 113s 152ms/step - accuracy: 0.6911 - loss: 0.8252 - precision_5: 0.7953 - recall_5: 0.5582 - val_accuracy: 0.6960 - val_loss: 0.7958 - val_precision_5: 0.7763 - val_recall_5: 0.6209 - learning_rate: 0.0010
Epoch 5/50
740/7

In [84]:
preds = model.predict(X_test)

print(classification_report(
    np.argmax(Y_test.values, axis=1),
    np.argmax(preds, axis=1),
    target_names=Y_test.columns.tolist()
))

463/463 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step
              precision    recall  f1-score   support

  Irrelevant       0.80      0.70      0.75      2696
    Negative       0.84      0.85      0.85      4380
     Neutral       0.79      0.73      0.76      3605
    Positive       0.76      0.87      0.81      4119

    accuracy                           0.80     14800
   macro avg       0.80      0.79      0.79     14800
weighted avg       0.80      0.80      0.80     14800

